# Using subclass API for dynamic models
* Sequential & Functional API : Declarative. Start by declaring which layers to use and how they are connected, then start feeding the model some data.
* Declarative method's advantages: can easily save, clone, and share the model, the structure of the models can be displayed and analyzed, easy to debug.
* However, declarative methods are static - cannot imploy loops, varying shapes, conditional branching, and other dynamic behaviors.
* Subclassing API can be used for the dynamic behaviors.

In [11]:
# Dataset:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_val_A, X_val_B = X_val[:, :5], X_val[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [12]:
# model using Subclass API
from tensorflow import keras

class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs): # create the layers
        super().__init__(**kwargs) # handles standard args(e.g., name)
        self.hidden1 = keras.layers.Dense(units, activation = activation)
        self.hidden2 = keras.layers.Dense(units, activation = activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)

    def call(self, inputs): # use the layers to perform computations
        # can use for-loops, if-statements, tf operations, etc in here.
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

model = WideAndDeepModel()
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer = "sgd") # the main output gets larger weight.
history = model.fit(
    [X_train_A, X_train_B], [y_train, y_train], epochs=5,
    validation_data=([X_val_A, X_val_B], [y_val, y_val]))

y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])    

Train on 11610 samples, validate on 3870 samples
Epoch 1/5
11610/11610 [==============================] - 1s 108us/sample - loss: 1.1580 - output_1_loss: 1.0327 - output_2_loss: 2.2836 - val_loss: 0.7642 - val_output_1_loss: 0.6891 - val_output_2_loss: 1.4388
Epoch 2/5
11610/11610 [==============================] - 1s 56us/sample - loss: 0.7993 - output_1_loss: 0.7536 - output_2_loss: 1.2106 - val_loss: 0.6221 - val_output_1_loss: 0.5702 - val_output_2_loss: 1.0877
Epoch 3/5
11610/11610 [==============================] - 1s 55us/sample - loss: 0.5778 - output_1_loss: 0.5315 - output_2_loss: 0.9954 - val_loss: 0.5661 - val_output_1_loss: 0.5248 - val_output_2_loss: 0.9369
Epoch 4/5
11610/11610 [==============================] - 1s 59us/sample - loss: 0.5400 - output_1_loss: 0.5015 - output_2_loss: 0.8857 - val_loss: 0.5502 - val_output_1_loss: 0.5167 - val_output_2_loss: 0.8498
Epoch 5/5
11610/11610 [==============================] - 1s 59us/sample - loss: 0.5093 - output_1_loss: 0.4764

In [13]:
# The above is equivalent to:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2) # for regularization
model = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

# Each output needs its own loss function - pass a list of loss functions.
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer = "sgd") # the main output gets larger weight.
history = model.fit(
    [X_train_A, X_train_B], [y_train, y_train], epochs=5,
    validation_data=([X_val_A, X_val_B], [y_val, y_val]))

y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])        

Train on 11610 samples, validate on 3870 samples
Epoch 1/5
11610/11610 [==============================] - 2s 146us/sample - loss: 1.2970 - main_output_loss: 1.1499 - aux_output_loss: 2.6167 - val_loss: 0.7074 - val_main_output_loss: 0.6436 - val_aux_output_loss: 1.2804
Epoch 2/5
11610/11610 [==============================] - 1s 60us/sample - loss: 0.6152 - main_output_loss: 0.5564 - aux_output_loss: 1.1447 - val_loss: 0.5771 - val_main_output_loss: 0.5209 - val_aux_output_loss: 1.0813
Epoch 3/5
11610/11610 [==============================] - 1s 60us/sample - loss: 0.6029 - main_output_loss: 0.5587 - aux_output_loss: 1.0015 - val_loss: 0.5360 - val_main_output_loss: 0.4901 - val_aux_output_loss: 0.9477
Epoch 4/5
11610/11610 [==============================] - 1s 58us/sample - loss: 0.5148 - main_output_loss: 0.4729 - aux_output_loss: 0.8918 - val_loss: 0.5215 - val_main_output_loss: 0.4839 - val_aux_output_loss: 0.8584
Epoch 5/5
11610/11610 [==============================] - 1s 59us/sampl